# Transfer raw files to USB

This script will treat folders created within 10 hours as new rawdata. It will then copy each folder over to their respective blocks on the USB. If blocks are missing, it will throw an error before asking to proceed. 

Note this is for the Host PC at TONI

In [ ]:
$sourceBase = "C:\Users\samhs\OneDrive\Documents\EYETRACKING"
$destinationBase = "D:\Powershellmove"
$currentDate = Get-Date
$blocks = 1..4
$missingBlocks = @()

foreach ($block in $blocks) {
    $sourcePath = Join-Path -Path $sourceBase -ChildPath ("FinalR" + $block + "D\results")
    $destinationPath = Join-Path -Path $destinationBase -ChildPath ("Encoding_Block" + $block)

    $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

    if ($rawdata.Count -eq 0) {
        $missingBlocks += $block
        continue
    }

    foreach ($dir in $rawdata) {
        $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
        $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
        $creationTime = $dir.CreationTime
        $userInput = $null

        if ($block -eq 1) {
            $userInput = Read-Host "Do you want to move directories from FinalR1D/results, FinalR2D/results, FinalR3D/results, and FinalR4D/results to their respective destinations on the USB? [Y/N]"
        }

        if ($userInput -eq "Y") {
            $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to $($destinationPath)"
        }
    }
}

if ($missingBlocks.Count -gt 0) {
    $missingBlockStr = $missingBlocks -join ", "
    $continueAnyway = Read-Host "Unexpected event: block(s) [$missingBlockStr] missing. Continue anyway? [Y/N]"
    if ($continueAnyway -ne "Y") {
        Write-Output "Operation aborted due to missing blocks."
        exit
    }
}
